In [ ]:
from lightweight import chat
from search import chat as search
from topics import topics
from rag import RAG
# from gpt import gpt

In [ ]:
prompt="Who is Lady Gaga?"
s=search(prompt)


In [ ]:
t=topics(s)


In [ ]:
c=chat("What is the most densley populated city in the world?")
c

In [9]:
r=RAG("What challenges is airbnb facing?")

100%|██████████| 4/4 [00:00<00:00, 1040.77it/s]
